# Kotlin **Asynchronous/Concurren** Programming **Best Practices**

In [3]:
// Importing some neccessary libs in jupyter notebook
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.5.2")
import kotlinx.coroutines.*
import java.util.concurrent.Executors
import kotlin.coroutines.CoroutineContext

In [4]:
// Use 'runBlocking' only at the entry of your program
fun main() = runBlocking {
    println("Hi")
}

In [10]:
// Try using 'suspend' functions to better organise your code

suspend fun fetchBackend() {
    // Fetch your backend
    println("fetched")
}
fun main() = runBlocking {
    fetchBackend()
}
main()

fetched


In [11]:
// Use 'coroutineScope' if you wanna 'launch' multiple requests
suspend fun fetchBackend() = coroutineScope {
    // Fetch your backend
    println("fetching started.")
    List(5) { i ->
        launch { println("end-point $i fetched") }
    }
    
}
fun main() = runBlocking {
    fetchBackend()
}
main()

fetching started.
end-point 0 fetched
end-point 1 fetched
end-point 2 fetched
end-point 3 fetched
end-point 4 fetched


[StandaloneCoroutine{Completed}@57f451b1, StandaloneCoroutine{Completed}@d2eed32, StandaloneCoroutine{Completed}@4cb433e5, StandaloneCoroutine{Completed}@40ef030b, StandaloneCoroutine{Completed}@3563ab7e]

In [21]:
/*
Use 'coroutineScope', 'async' and 'awaitAll' if you wanna launch multiple requests
and get the result.
*/

suspend fun fetchBackend() = coroutineScope {
    // Fetch your backend
    println("fetching started.")
    val reqs = List(5) { i ->
        async { 
            println("end-point $i fetched") 
            i*i
        }
    }
    val res = reqs.awaitAll()
    println("fetching finished.")
    return@coroutineScope res
}
fun main() = runBlocking {
    val res = fetchBackend()
    println("fetch result: $res")
}
main()

fetching started.
end-point 0 fetched
end-point 1 fetched
end-point 2 fetched
end-point 3 fetched
end-point 4 fetched
fetching finished.
fetch result: [0, 1, 4, 9, 16]


In [22]:
/* 
Use 'withContext' to create a coroutine scope 
with is run on the given dispatcher.
*/

suspend fun fetchBackend() = withContext(Dispatchers.IO) {
    // Fetch your backend on IO threads pool
    println("fetching started.")
    val reqs = List(5) { i ->
        async { 
            println("end-point $i fetched") 
            i*i
        }
    }
    val res = reqs.awaitAll()
    println("fetching finished.")
    return@withContext res
}
fun main() = runBlocking {
    val res = fetchBackend()
    println("fetch result: $res")
}
main()

fetching started.
end-point 0 fetched
end-point 1 fetched
end-point 2 fetched
end-point 3 fetched
end-point 4 fetched
fetching finished.
fetch result: [0, 1, 4, 9, 16]


In [46]:
// Only create custom 'CoroutineScope' when it is neccessary.

val threadNo = 10
val contextJob = Job()
val contextDispatcher = Executors.newFixedThreadPool(threadNo).asCoroutineDispatcher()
val context: CoroutineContext = contextDispatcher + contextJob + CoroutineName("Powerful but heavy Scope")
val myScope = CoroutineScope(context)

suspend fun doImportant(): List<Int> {
    // Fetch your backend
    println("Doing important started.")
    val reqs = List(100) { i ->
        myScope.async { 
            delay(1000)
            print("imp{$i} done, ") 
            i*i
        }
    }
    val res = reqs.awaitAll()
    println("\nDoing important finished.")
    return res
}
fun main() = runBlocking {
    val res = doImportant()
    myScope.cancel()
    println("Doing important result: ${res.sum()}")
}
main()                           

Doing important started.
imp{0} done, imp{11} done, imp{12} done, imp{10} done, imp{13} done, imp{14} done, imp{15} done, imp{16} done, imp{17} done, imp{18} done, imp{19} done, imp{20} done, imp{21} done, imp{22} done, imp{23} done, imp{24} done, imp{25} done, imp{26} done, imp{27} done, imp{33} done, imp{28} done, imp{29} done, imp{30} done, imp{34} done, imp{31} done, imp{32} done, imp{35} done, imp{36} done, imp{37} done, imp{38} done, imp{39} done, imp{40} done, imp{41} done, imp{43} done, imp{42} done, imp{44} done, imp{45} done, imp{46} done, imp{47} done, imp{48} done, imp{49} done, imp{50} done, imp{51} done, imp{52} done, imp{53} done, imp{54} done, imp{55} done, imp{56} done, imp{57} done, imp{58} done, imp{59} done, imp{60} done, imp{61} done, imp{62} done, imp{63} done, imp{66} done, imp{64} done, imp{65} done, imp{67} done, imp{68} done, imp{69} done, imp{70} done, imp{71} done, imp{72} done, imp{73} done, imp{82} done, imp{74} done, imp{75} done, imp{76} done, imp{77} do

In [47]:
// Or it is better to do it using 'withContext' as:

val threadNo = 10
val myDispatcher: CoroutineContext = Executors.newFixedThreadPool(threadNo).asCoroutineDispatcher()

suspend fun doImportant(): List<Int> = withContext(myDispatcher) {
    // Fetch your backend
    println("Doing important started.")
    val reqs = List(100) { i ->
        async { 
            delay(1000)
            print("imp{$i} done, ") 
            i*i
        }
    }
    val res = reqs.awaitAll()
    println("\nDoing important finished.")
    return@withContext res
}
fun main() = runBlocking {
    val res = doImportant()
    myDispatcher.cancel()
    println("Doing important result: ${res.sum()}")
}
main()       

Doing important started.
imp{10} done, imp{2} done, imp{3} done, imp{11} done, imp{0} done, imp{12} done, imp{9} done, imp{13} done, imp{1} done, imp{14} done, imp{19} done, imp{15} done, imp{16} done, imp{17} done, imp{18} done, imp{20} done, imp{21} done, imp{22} done, imp{23} done, imp{24} done, imp{4} done, imp{25} done, imp{27} done, imp{26} done, imp{29} done, imp{28} done, imp{30} done, imp{31} done, imp{32} done, imp{33} done, imp{34} done, imp{35} done, imp{36} done, imp{37} done, imp{38} done, imp{39} done, imp{40} done, imp{41} done, imp{42} done, imp{43} done, imp{44} done, imp{45} done, imp{46} done, imp{47} done, imp{48} done, imp{49} done, imp{50} done, imp{51} done, imp{52} done, imp{53} done, imp{54} done, imp{55} done, imp{59} done, imp{56} done, imp{57} done, imp{58} done, imp{60} done, imp{61} done, imp{62} done, imp{63} done, imp{64} done, imp{65} done, imp{66} done, imp{67} done, imp{68} done, imp{69} done, imp{70} done, imp{71} done, imp{72} done, imp{73} done, i

In [ ]:
// Consider finilizing resources when a cancellation request might be received

